In [1]:
import numpy as np
import pandas as pd
from datetime import date

import rdkit.Chem
from molmass import Formula
from rdkit import Chem
from rdkit.Chem import Descriptors

from rdkit.Chem import AllChem as Chem
from rdkit.Chem.MolStandardize import rdMolStandardize

from pubchempy import get_compounds, Compound


In [ ]:
import logging

logging.getLogger('pubchempy').setLevel(logging.DEBUG)

In [ ]:
# define all variables
lib_id = "mce"

# usually empty unless, e.g., second measurement or other parameters
# always ends with underscore _
prefix = "100AGC_60000Res_"
instrument_method = r"C:\Xcalibur\methods\Corinna_Brungs\Library6_100AGC_60000Res_MS5_POS_mz115-2000"

plates = ["1D1", "1D2", "1D3"]
plate_id_header = "mixed_location_plate1"

# plates are inserted into the BLUE B compartment
plate_loc_in_autosampler = "B"

# final values
unique_id_header = "lib_plate_well"
raw_filename = "raw_filename"

library_file = "data/{}_library.csv".format(lib_id)

## Import library

In [ ]:
lib_df = pd.read_csv(library_file, sep="\t")
lib_df


## Add unique column with internal ID and well location
Use internal ID of plate and then library ID

In [ ]:
def exact_mass(formula):
    try:
        clean = formula.split(".")[0]
        return Formula(clean).isotope.mass
    except:
        return np.NAN



## Get exact mass from cleaned SMILES

In [ ]:
# returns canonical smiles
def mol_to_canon_smiles(mol):
    try:
        return Chem.MolToSmiles(mol, isomericSmiles=True)
    except:
        return None


# def smi_to_canon_smiles(smi):
#     try:
#         return Chem.MolToSmiles(Chem.MolFromSmiles(smi), isomericSmiles=False)
#     except:
#         pass

uncharger = rdMolStandardize.Uncharger()


# smiles_stats = {'n_dots': Counter(), 'charge': Counter(), 'invalid_smiles': []}


def cleaned_mol(smiles: str):
    original_input = smiles
    try:
        # find the longest smiles that might be the main molecule
        # for smiles that contain the salt partner etc
        split_smiles = str(smiles).split('.')
        if len(split_smiles) > 1:
            # smiles_stats['n_dots'][len(split_smiles)-1] += 1
            smiles = max(split_smiles, key=len)
        else:
            smiles = split_smiles[0]

        mol = Chem.MolFromSmiles(smiles)
        charge = Chem.GetFormalCharge(mol)
        if abs(charge) > 0:
            # smiles_stats['charge'][charge] += 1
            mol = uncharger.uncharge(mol)

        if mol is None:
            return mol_from_pepseq(original_input)
        else:
            return mol
    except:
        return mol_from_pepseq(original_input)


def mol_from_pepseq(original_input):
    # read protein seq
    try:
        sequence = str(original_input).replace("[", "").replace("]", "").replace(" (TFA salt)", "")
        return Chem.MolFromSequence(sequence)
    except:
        return None


def exact_mass_from_mol(mol):
    try:
        # canonical
        return Descriptors.ExactMolWt(mol)
    except:
        return None

# def exact_mass_from_smiles(smiles: str):
#     try:
#         # find the longest smiles that might be the main molecule
#         # for smiles that contain the salt partner etc
#         split_smiles = smiles.split('.')
#         if len(split_smiles) > 1:
#             # smiles_stats['n_dots'][len(split_smiles)-1] += 1
#             smiles = max(split_smiles, key=len)
#         else:
#             smiles = split_smiles[0]
#
#
#         # for those smiles provided as salts (e.g., .Na+) add H+ until charge is neutral
#         # if charge is neutral already (N+ and PO-) keep both charges
#         mol = Chem.MolFromSmiles(smiles)
#         charge = Chem.GetFormalCharge(mol)
#         if abs(charge) > 0:
#             # smiles_stats['charge'][charge] += 1
#             mol = uncharger.uncharge(mol)
#
#         # canonical
#         return Descriptors.ExactMolWt(mol)
#     except:
#         return np.NAN

In [ ]:
# define file names
lib_df[unique_id_header] = ["pluskal_{}_{}".format(lib_id, plate_id) for plate_id in lib_df[plate_id_header]]
# lib_df[raw_filename] = ["{}_{}{}".format(current_date, prefix, unique_id) for unique_id in lib_df[unique_id_header]]


electron_mass = 0.00054857
mzh = exact_mass("H") - electron_mass
mzna = exact_mass("Na") - electron_mass

# define exact mass
if not "exact_mass" in lib_df:
    lib_df["exact_mass"] = [exact_mass(formula) for formula in lib_df["Formula"]]
    lib_df["mz_h"] = lib_df["exact_mass"] + mzh
    lib_df["mz_na"] = lib_df["exact_mass"] + mzna

# from smiles
mols = [cleaned_mol(smiles) for smiles in lib_df["Smiles"]]
lib_df["cleaned_smiles"] = [mol_to_canon_smiles(mol) for mol in mols]
lib_df["exact_mass_smiles"] = [exact_mass_from_mol(mol) for mol in mols]
lib_df["mz_h_smiles"] = lib_df["exact_mass_smiles"] + mzh
lib_df["mz_na_smiles"] = lib_df["exact_mass_smiles"] + mzna

lib_df["mass_matches"] = [abs(a - b) < 0.01 for a, b in zip(lib_df["exact_mass_smiles"], lib_df["exact_mass"])]

# lib_df.to_csv("data/lib_formatted_{}.csv".format(lib_id), sep="\t", index=False)

lib_df

## Getting compound information from CAS or Name (CAS to PubChem)

In [ ]:
def compound_score(comp: Compound):
    smiles = comp.canonical_smiles
    if not smiles:
        return 0
    return 1000 - str(smiles).count(".")


def search_pubchem_by_name(name_or_cas: str) -> Compound | None:
    """
    In pubchem many entries contain the cas as an alternative name - so searching for cas in name works often

    :param name_or_cas: input name or cas
    :return: first compound or None
    """
    if name_or_cas == "NaN":
        return None
    compounds = get_compounds(name_or_cas, "name")
    if not compounds:
        logging.info("cas:{} had NO entries".format(name_or_cas))
        return None
    else:
        compounds.sort(key=lambda comp: compound_score(comp), reverse=True)
        return compounds[0]

In [ ]:
compounds = [search_pubchem_by_name(str(cas)) if not pd.isnull(cas) else np.NAN for cas in lib_df["CAS No."]]

In [ ]:
compounds = [search_pubchem_by_name(str(name)) if pd.isnull(comp) else comp for comp, name in
             zip(compounds, lib_df["Product Name"])]
# only one compound was found as CAS-
compounds = [search_pubchem_by_name("CAS-{}".format(cas)) if pd.isnull(comp) else comp for comp, cas in
             zip(compounds, lib_df["CAS No."])]


In [ ]:
lib_df["PubChemID"] = pd.array([compound.cid if not pd.isnull(compound) else np.NAN for compound in compounds],
                               dtype=pd.Int64Dtype())
lib_df["isomeric_smiles"] = [compound.isomeric_smiles if not pd.isnull(compound) else np.NAN for compound in compounds]
lib_df["canonical_smiles"] = [compound.canonical_smiles if not pd.isnull(compound) else np.NAN for compound in
                              compounds]
lib_df

## Cleaning PubChem Smiles and calculating exact mass

In [ ]:
electron_mass = 0.00054857
mzh = exact_mass("H") - electron_mass
mzna = exact_mass("Na") - electron_mass

# from smiles
mols = [cleaned_mol(smiles) if not pd.isnull(smiles) else np.NAN for smiles in lib_df["isomeric_smiles"]]
lib_df["cleaned_psmiles"] = [mol_to_canon_smiles(mol) for mol in mols]
lib_df["exact_mass_psmiles"] = [exact_mass_from_mol(mol) for mol in mols]
lib_df["mz_h_psmiles"] = lib_df["exact_mass_psmiles"] + mzh
lib_df["mz_na_psmiles"] = lib_df["exact_mass_psmiles"] + mzna

lib_df["mce_smiles_vs_pubchem_smiles"] = [abs(a - b) < 0.01 for a, b in
                                          zip(lib_df["exact_mass_smiles"], lib_df["exact_mass_psmiles"])]

lib_df.to_csv("data/lib_formatted_pubchem_{}.csv".format(lib_id), sep="\t", index=False)

lib_df

In [ ]:
lib_df[lib_df["exact_mass_smiles"].isna()]

### Creating one cleaned SMILES column, compounds can have multiple entries if provided SMILES and PubChem SMILES are different

In [ ]:
lib_df = pd.read_csv("data/lib_formatted_pubchem_mce.csv", sep="\t")
add_df = pd.read_csv("data/lib_formatted_mce_add_compounds.csv", sep="/t")

In [ ]:
lib1_df = lib_df[
    ["Cat. No.", "Product Name", "Synonyms", "CAS No.", "Smiles", "PubChemID", "isomeric_smiles", "canonical_smiles",
     "lib_plate_well", "URL", "Target", "Information", "Pathway", "Research Area", "Clinical Information"]].copy()

lib1_df["Source"] = "MCE"
lib1_df

In [ ]:
lib2_df = lib_df[
    ["Cat. No.", "Product Name", "Synonyms", "CAS No.", "Smiles", "PubChemID", "isomeric_smiles", "canonical_smiles",
     "lib_plate_well", "URL", "Target", "Information", "Pathway", "Research Area", "Clinical Information"]].copy()
lib2_df["Smiles"] = lib2_df["canonical_smiles"]
lib2_df["Source"] = "PubChem"
lib2_df

In [ ]:
merged_df = pd.concat([lib2_df, lib1_df], ignore_index=True, sort=False)
merged_df

In [ ]:
df = pd.read_csv("data/test_metadata_cleaned.tsv", sep="\t")

In [ ]:
synonyms = ['4egi-1', '315706-13-9', 'CHEMBL254578', 'UNIi ;-H57R:EU3DHP', 'H57REU3DHP', '4EGI1', '4EGI 1',
            'SCHEMBL3334288', 'alpha-(2-(4-(3,4-Dichlorophenyl)-2-thiazolyl)hydrazinylidene)-2-nitrobenzenepropano',
            "UNII-123124dawdawd"]

import re

# [name for name in synonyms if "UNII" in name]
gen = (re.sub('[ .;:\-]|UNII', '', name.upper()) for name in synonyms if "UNII" in name.upper())

next(gen)
# re.sub('[ .;:\-]|UNII', '', next(gen))

In [193]:
import xml.etree.ElementTree as ET
import csv

tree = ET.parse("data/drugbank_database.xml")
root = tree.getroot()
print(root)

<Element 'drugbank' at 0x000002A36C1B0310>


In [203]:
name = node.attrib.get("type")
print(name)

biotech


In [ ]:
# ET.dump(tree)

In [204]:
rows = []
properties = ["SMILES", "InChI", "InChIKey"]
resources = ["PubChem Substance", "ChEMBL"]

def from_xml_node_path(node, path_list):
    if node is None:
        return None
    if len(path_list) == 1:
        return from_xml_node(node, path_list[0])
    else:
        return from_xml_node_path(node.find(path_list[0]), path_list[1:])


def from_xml_node(node, node_id):
    if node is None:
        return None
    subnode = node.find(node_id)
    return subnode.text if subnode is not None else None


def from_xml_attribute(node, node_id):
    return node.attrib.get(node_id) if node else None



resources = ["PubChem Substance", "ChEMBL"]

for node in root:
    groups_node = node.find("groups")

    external = []
    atc_node = None
    mode_of_action = None
    food_interaction = None
    targets_node = None
    targets = None

    smiles = None
    inchikey = None
    try:
        calculated_properties_node = node.find("calculated-properties")
        if calculated_properties_node:
            smiles = next((from_xml_node(prop, "value") for prop in calculated_properties_node if from_xml_node(prop, "kind") == "SMILES"), None)
            inchikey = next((from_xml_node(prop, "value") for prop in calculated_properties_node if from_xml_node(prop, "kind") == "InChIKey"), None)
    except: pass

    try:
        atc_node = node.find("atc-codes").find("atc-code")
        mode_of_action = ", ".join([level.text for level in atc_node])
    except: pass

    try:
        food_interaction_node = node.find("food-interactions")
        food_interaction = ", ".join([interaction.text for interaction in food_interaction_node])
    except: pass

    try:
        external_identifiers_node = node.find("external-identifiers")
        external = [from_xml_node(id_node, "identifier") for id_node in external_identifiers_node if
                    from_xml_node(id_node, "resource") in resources]
    except: pass

    targets = None
    try:
        targets_node = node.find("targets")
        targets = [(from_xml_node(target, "name"), from_xml_node(target, "organism"), from_xml_node_path(target, path_list=["actions", "action"]))  for target in targets_node]
        targets = ", ".join(["({})".format("; ".join(target)) for target in targets])
    except: pass

    rows.append({"drugbank_id": from_xml_node(node, "drugbank-id"),
                     "name": from_xml_node(node, "name"),
                     "chembl_id": next(filter(lambda d: str(d).startswith("CHEMBL"), external), None),
                     # currently we only extract pubchem and chembl, that's why this works (excluding chembl, only shows pubchem)
                     "pubchem_cid": next(filter(lambda d: not str(d).startswith("CHEMBL"), external), None),
                     "cas": from_xml_node(node, "cas-number"),
                     "unii": from_xml_node(node, "unii"),
                     "smiles": smiles,
                     # "inchi": from_xml_node(node, ""),
                     "inchikey": inchikey,
                    "type": from_xml_attribute(node, "type"),

                     "approved": from_xml_node(groups_node, "group"),
                     "indication": from_xml_node(node, "indication"),
                     "pharmacodynamics": from_xml_node(node, "pharmacodynamics"),
                     "mechanism_of_action": from_xml_node(node, "mechanism-of-action"),
                     "toxicity": from_xml_node(node, "toxicity"),
                     "metabolism": from_xml_node(node, "metabolism"),
                     "absorption": from_xml_node(node, "absorption"),
                     "half_life": from_xml_node(node, "half-life"),
                     "route_of_elimination": from_xml_node(node, "route-of-elimination"),
                     "clearance": from_xml_node(node, "clearance"),
                     "atc_code": from_xml_attribute(atc_node, "code"),
                     "mode_of_action": mode_of_action,
                     "food_interaction": food_interaction,
                     "targets": targets,
                     })

out_df = pd.DataFrame(rows)
out_df

# out_df.to_csv("data/drugbank_test.tsv", sep="\t", index=False)

,drugbank_id,name,chembl_id,pubchem_cid,cas,unii,smiles,inchikey,type,approved,...,toxicity,metabolism,absorption,half_life,route_of_elimination,clearance,atc_code,mode_of_action,food_interaction,targets
0,DB00001,Lepirudin,CHEMBL1201666,46507011,138068-37-8,Y43GF64R34,None,None,biotech,approved,...,"In case of overdose (eg, suggested by excessiv...",Lepirudin is thought to be metabolized by rele...,Bioavailability is 100% following injection.,Approximately 1.3 hours,Lepirudin is thought to be metabolized by rele...,* 164 ml/min [Healthy 18-60 yrs]\r\n* 139 ml/m...,B01AE02,"Direct thrombin inhibitors, ANTITHROMBOTIC AGE...",Avoid herbs and supplements with anticoagulant...,(Prothrombin; Humans; inhibitor)
1,DB00002,Cetuximab,CHEMBL1201577,46507042,205923-56-4,PQX0D8J21J,None,None,biotech,approved,...,The intravenous LD<sub>50</sub> is > 300 mg/kg...,"Like other monoclonal antibodies, cetuximab is...",After administration of a 400 mg/m<sup>2</sup>...,After administration of a 400 mg/m<sup>2</sup>...,There is limited information available.,In patients with recurrent and/or metastatic s...,L01XC06,"Monoclonal antibodies, OTHER ANTINEOPLASTIC AG...",,(Epidermal growth factor receptor; Humans; bin...
2,DB00003,Dornase alfa,CHEMBL1201431,46507792,143831-71-4,953A26OA1Y,None,None,biotech,approved,...,Adverse reactions occur at a frequency of < 1/...,While no conclusive studies have yet been publ...,Studies in rats and monkeys after inhalation o...,None,None,"Studies in rats indicate that, following aeros...",R05CB13,"Mucolytics, EXPECTORANTS, EXCL. COMBINATIONS W...",,"[(DNA, Humans, None)]"
3,DB00004,Denileukin diftitox,CHEMBL1201550,46506950,173146-27-5,25E79B5CTM,None,None,biotech,approved,...,None,None,None,70-80 min,None,* 0.6 - 2.0 mL/min/kg [Lymphoma],L01XX29,"Other antineoplastic agents, OTHER ANTINEOPLAS...",,"[(Interleukin-2 receptor subunit alpha, Humans..."
4,DB00005,Etanercept,CHEMBL1201572,46506732,185243-69-0,OP401G7OJC,None,None,biotech,approved,...,None,"As etanercept is a fusion protein antibody, it...",Population pharmacokinetic modeling in adults ...,Etanercept has a mean half-life of elimination...,None,Etanercept has a mean apparent clearance of 16...,L04AB01,Tumor necrosis factor alpha (TNF-α) inhibitors...,,"(Tumor necrosis factor; Humans; antibody), (Hi..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14589,DB16742,RP-67580,None,None,135911-02-3,49U9M41BGY,None,None,small molecule,experimental,...,None,None,None,None,None,None,None,None,,
14590,DB16743,Nolpitantium chloride,None,None,153050-21-6,22O6XI63E0,None,None,small molecule,experimental,...,None,None,None,None,None,None,None,None,,
14591,DB16744,CP-96345,None,None,132746-60-2,W22ILA2I52,None,None,small molecule,experimental,...,None,None,None,None,None,None,None,None,,
14592,DB16745,PXT 3003,None,None,1467047-91-1,None,CCOC(=O)C1=CC(NCC2=CC=CC=C2)=C(NC2CCCCC2)N=C1,WRUIDZKNUAHKTR-UHFFFAOYSA-N,small molecule,investigational,...,None,None,None,None,None,None,None,None,,


In [ ]:
from xml.dom.minidom import parse, parseString

document = parse('data/drugbank_small.xml')

In [211]:
df = pd.read_csv("data/test_metadata_cleaned.tsv", sep="\t")
df[["Product Name", "pubchem_cid", "pubchem_cid_parent", "inchi_key", "split_inchi_key", "drugbank_inchi_key"]]

,Product Name,pubchem_cid,pubchem_cid_parent,inchi_key,split_inchi_key,drugbank_inchi_key
0,TG003,1893668,1893668,BGVLELSCIHASRV-QPEQYQDCSA-N,BGVLELSCIHASRV,NaN
1,Fostemsavir Tris,46892186,11319217,SWMDAPWAQQTBOG-UHFFFAOYSA-N,SWMDAPWAQQTBOG,SWMDAPWAQQTBOG-UHFFFAOYSA-N
2,NQO1 substrate,138454764,138454764,PZUSGRHVYDQLHR-UHFFFAOYSA-N,PZUSGRHVYDQLHR,NaN
3,ALK inhibitor 1,24857689,24857689,FTSDLONCFCQDGA-UHFFFAOYSA-N,FTSDLONCFCQDGA,NaN
4,CCT007093,2314623,2314623,KPFZCKDPBMGECB-WGDLNXRISA-N,KPFZCKDPBMGECB,NaN
5,Brevianamide F,181567,181567,RYFZBPVMVYTEKZ-KBPBESRZSA-N,RYFZBPVMVYTEKZ,NaN
6,PF-04691502,25033539,25033539,XDLYKKIQACFMJG-UHFFFAOYSA-N,XDLYKKIQACFMJG,XDLYKKIQACFMJG-WKILWMFISA-N
7,CH5132799,49784945,49784945,JEGHXKRHKHPBJD-UHFFFAOYSA-N,JEGHXKRHKHPBJD,JEGHXKRHKHPBJD-UHFFFAOYSA-N
8,IPSU,56970858,56970858,PCMHOSYCWRRHTG-UHFFFAOYSA-N,PCMHOSYCWRRHTG,NaN
9,(Z)-4EGI-1,6911989,6911989,KFRKRECSIYXARE-HMAPJEAMSA-N,KFRKRECSIYXARE,NaN


In [ ]:
next((d for d in df[df["name"]=="Tolmetin"]["unii"]), None)

In [9]:
import os

In [10]:
structures.* from structures,approval where structures.id = approval.struct_id and approval.type = 'FDA'

SyntaxError: invalid syntax (3982089494.py, line 1)

In [12]:
dc_df = pd.read_csv("data/drugcentral_smiles.tsv", sep="\t")

In [ ]:
select structures.* from structures;